In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB

In [ ]:
url = 'https://raw.githubusercontent.com/chlaudiah/Sentiment-Classification-FD-Reviews/master/dataset.csv'

In [ ]:
df = pd.read_csv(url) #read data
df.head() #munculkan data

,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label
0,A001,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,Positive
1,A002,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,Positive
2,A003,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,Positive
3,A004,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,Positive
4,A005,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,Positive


In [ ]:
df['Label'] = df['Label'].replace(['negative','positive'],['Negative','Positive'])
df

,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label
0,A001,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,Positive
1,A002,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,Positive
2,A003,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,Positive
3,A004,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,Positive
4,A005,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,Positive
...,...,...,...,...,...,...
495,A496,Acnes: Creamy Wash,Dry,Aku beli ini karna baca blog female daily 3 cl...,1,Negative
496,A497,Acnes: Creamy Wash,Dry,"aku pake ini udh ngabisin 1 botol, saat aku je...",1,Negative
497,A498,Acnes: Creamy Wash,Dry,Facial Wash yg aku beli pas awal2 mulai jerawa...,1,Negative
498,A499,Acnes: Creamy Wash,Dry,Aku pernah beli ini dengan harapan bisa bantu ...,1,Negative


In [ ]:
df.shape

(500, 6)

In [ ]:
df.isnull().any(axis=1).sum()

0

In [ ]:
df = df.drop(columns=['ID Data'])

In [ ]:
df['Label'].value_counts()

Positive    250
Negative    250
Name: Label, dtype: int64

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

lbl       = le.fit_transform(df['Label']) 

df['Label'] = lbl
df

,Nama Produk,Jenis Kulit,Review Produk,Rating,Label
0,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,1
1,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,1
2,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,1
3,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,1
4,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,1
...,...,...,...,...,...
495,Acnes: Creamy Wash,Dry,Aku beli ini karna baca blog female daily 3 cl...,1,0
496,Acnes: Creamy Wash,Dry,"aku pake ini udh ngabisin 1 botol, saat aku je...",1,0
497,Acnes: Creamy Wash,Dry,Facial Wash yg aku beli pas awal2 mulai jerawa...,1,0
498,Acnes: Creamy Wash,Dry,Aku pernah beli ini dengan harapan bisa bantu ...,1,0


In [ ]:
import en_core_web_sm
import string
from spacy.lang.id.stop_words import STOP_WORDS
punct = string.punctuation
#nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()
stopwords = list(STOP_WORDS)

In [ ]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import re

tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

X = df['Review Produk']
#X = X.lower()
#X = [re.sub('[^a-zA-Z 0-9]', ' ', X)]
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


#print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred, normalize = True)
accuracy

0.88

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.92      0.89        52
           1       0.91      0.83      0.87        48

    accuracy                           0.88       100
   macro avg       0.88      0.88      0.88       100
weighted avg       0.88      0.88      0.88       100



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[69,  7],
       [12, 62]])

In [ ]:
# with the following sentence you can get a mask of the items bad classified
mask = np.logical_not(np.equal(y_test, y_pred))
# Now you can use the mask to see the elements bad classified:
print(f"Elements wrong classified: {X_test[mask]}")
print(f"Prediction by the model for each of those elements: {y_pred[mask]}")
print(f"Actual value for each of those elements: {np.asarray(y_test)[mask]}")

Elements wrong classified: 487    Sebelumnya pakai Hada Labo mild peeling cleans...
29     beli karena ini lagi sale terus banyak banget ...
394    coba FW ini waktu itu dari rangkaian kit nya.....
345    gk recom untuk dry skin karena habis dibilas a...
106    Awalnya coba yg starter pack kecil (anti agein...
250    akuu sukaa banget sama yang satu ini ? parahhh...
74     Sukaaa banget sama produk ini!! Lembut di kuli...
310    Pernah satu kali beli Cleanser ini karena kepe...
376    Ngebersihin pake banget ?? cuma di aku kok ber...
499    Beli ini waktu awal kuliah pas banyak jerawat....
373    Enak banget cuci muka pake ft gentle cleanser ...
342    Uda bertahun2 nyoba anti-acne facial wash berb...
495    Aku beli ini karna baca blog female daily 3 cl...
405    Wardah emang terkenal dengan brand HALAL, mura...
228    wanginya krg enak, efeknya krg bagus di muka, ...
471    salah satu product acnes yang nggak begitu bag...
438    So far pake ini yaa, superbbbb bagus. Gara2nya...
360 

In [ ]:
misclassified_samples = X_test[y_test != y_pred]
misclassified_samples

487    Sebelumnya pakai Hada Labo mild peeling cleans...
29     beli karena ini lagi sale terus banyak banget ...
394    coba FW ini waktu itu dari rangkaian kit nya.....
345    gk recom untuk dry skin karena habis dibilas a...
106    Awalnya coba yg starter pack kecil (anti agein...
250    akuu sukaa banget sama yang satu ini ? parahhh...
74     Sukaaa banget sama produk ini!! Lembut di kuli...
310    Pernah satu kali beli Cleanser ini karena kepe...
376    Ngebersihin pake banget ?? cuma di aku kok ber...
499    Beli ini waktu awal kuliah pas banyak jerawat....
373    Enak banget cuci muka pake ft gentle cleanser ...
342    Uda bertahun2 nyoba anti-acne facial wash berb...
495    Aku beli ini karna baca blog female daily 3 cl...
405    Wardah emang terkenal dengan brand HALAL, mura...
228    wanginya krg enak, efeknya krg bagus di muka, ...
471    salah satu product acnes yang nggak begitu bag...
438    So far pake ini yaa, superbbbb bagus. Gara2nya...
360    aku dapat sample ini saa